In [3]:
#titanic_kfp_pipeline.ipynb
#Copyright 2020 Google LLC. 
#This software is provided as-is, without warranty or representation for any use or purpose. 
#Your use of it is subject to your agreements with Google.
#Author: whjang@google.com

PROJECT_ID = 'kubeflow-test'
IMAGE_PREFIX = 'whjang-titanic'
PREPROC_DIR = 'preprocess'
TRAIN_DIR = 'train'

WORK_BUCKET = 'gs://kubeflow-test-280609-kubeflowpipelines-default'
RAW_CSV_GCS_URI = WORK_BUCKET + '/rawdata/train.csv'
PREPROC_CSV_GCS_URI = WORK_BUCKET + '/preprocdata/processed_train.csv'
ACC_CSV_GCS_URI = WORK_BUCKET + '/latestacc/accuracy.csv'
MODEL_PKL_GCS_URI = WORK_BUCKET + '/model/model.pkl'
STAGE_GCS_FOLDER = WORK_BUCKET + '/stage'

AIPJOB_TRAINER_GCS_PATH = WORK_BUCKET + '/train/titanic_train.tar.gz'
AIPJOB_OUTPUT_GCS_PATH = WORK_BUCKET + '/train/output/'

import os
os.environ["PROJECT_ID"] = PROJECT_ID
os.environ["IMAGE_PREFIX"] = IMAGE_PREFIX
os.environ["PREPROC_DIR"] = PREPROC_DIR
os.environ["TRAIN_DIR"] = TRAIN_DIR
os.environ["WORK_BUCKET"] = WORK_BUCKET
os.environ["RAW_CSV_GCS_URI"] = RAW_CSV_GCS_URI
os.environ["PREPROC_CSV_GCS_URI"] = PREPROC_CSV_GCS_URI
os.environ["ACC_CSV_GCS_URI"] = ACC_CSV_GCS_URI
os.environ["MODEL_PKL_GCS_URI"] = MODEL_PKL_GCS_URI
os.environ["STAGE_GCS_FOLDER"] = STAGE_GCS_FOLDER
os.environ["AIPJOB_TRAINER_GCS_PATH"] = AIPJOB_TRAINER_GCS_PATH
os.environ["AIPJOB_OUTPUT_GCS_PATH"] = AIPJOB_OUTPUT_GCS_PATH

In [38]:
!docker build -t $IMAGE_PREFIX-$PREPROC_DIR $PREPROC_DIR/.
!docker tag $IMAGE_PREFIX-$PREPROC_DIR:latest docker.io/insoopark/$IMAGE_PREFIX-$PREPROC_DIR:latest
!docker push docker.io/insoopark/$IMAGE_PREFIX-$PREPROC_DIR:latest

Sending build context to Docker daemon  8.704kB
Step 1/8 : FROM frolvlad/alpine-python-machinelearning
 ---> 9e141b2fee53
Step 2/8 : RUN pip install --upgrade gcsfs
 ---> Using cache
 ---> 144dc6ed561c
Step 3/8 : RUN pip install --upgrade argparse
 ---> Using cache
 ---> 768a977c9c9f
Step 4/8 : ENV PYTHONUNBUFFERED 1
 ---> Using cache
 ---> 6e19eedd5829
Step 5/8 : RUN mkdir -p /titanic/src
 ---> Using cache
 ---> 9a31dbd1a94c
Step 6/8 : COPY . /titanic/src
 ---> 098f2f6ee783
Step 7/8 : WORKDIR /titanic/src
 ---> Running in c7ffeaa402be
Removing intermediate container c7ffeaa402be
 ---> 238e00d97d98
Step 8/8 : ENTRYPOINT ["python", "titanic_preprocess.py"]
 ---> Running in 096314c4755a
Removing intermediate container 096314c4755a
 ---> 1e8fc6a5f7b5
Successfully built 1e8fc6a5f7b5
Successfully tagged whjang-titanic-preprocess:latest
The push refers to repository [docker.io/insoopark/whjang-titanic-preprocess]

9a5aeb69: Preparing 
ec50a8b8: Preparing 
b3466d13: Preparing 
8ff39333: Prepa

In [39]:
!docker run docker.io/insoopark/$IMAGE_PREFIX-$PREPROC_DIR:latest --raw_csv_gcs_uri $RAW_CSV_GCS_URI --preproc_csv_gcs_uri $PREPROC_CSV_GCS_URI

Preprocessing Titanic Data
   Pclass  SibSp  Sex_female  Sex_male  Survived
0       3      1           0         1         0
1       1      1           1         0         1
2       3      0           1         0         1
3       1      1           1         0         1
4       3      0           0         1         0


In [ ]:
!docker build -t $IMAGE_PREFIX-$TRAIN_DIR $TRAIN_DIR/.
!docker tag $IMAGE_PREFIX-$TRAIN_DIR:latest gcr.io/$PROJECT_ID/$IMAGE_PREFIX-$TRAIN_DIR:latest
!docker push docker.io/insoopark/$IMAGE_PREFIX-$TRAIN_DIR:latest
!docker run docker.io/insoopark/$IMAGE_PREFIX-$TRAIN_DIR:latest \
--preproc_csv_gcs_uri $PREPROC_CSV_GCS_URI \
--model_pkl_gcs_uri $MODEL_PKL_GCS_URI \
--acc_csv_gcs_uri $ACC_CSV_GCS_URI \
--min_acc_progress 0.000001